In [30]:
from pytube import YouTube
import os
import shutil

curr_dir = "/home/sean/Documents/experiments/stage-mix-generator/"
download_dir = os.path.join(curr_dir, 'temp')

In [31]:
# get videos via youtube api
lyrics_video_url = "https://www.youtube.com/watch?v=mIXR-7u2Kas"
stage_video_urls = [
    "https://www.youtube.com/watch?v=kda_53A0ZpI",
    "https://www.youtube.com/watch?v=zzq8uI1BnJk",
    "https://www.youtube.com/watch?v=Q65bS5EyX5I",
    "https://www.youtube.com/watch?v=QsV25Q6VldA"
]

In [35]:
# download videos

ydl_opts = {     
    'outtmpl': os.path.join(download_dir, '%(id)s.%(ext)s')
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([lyrics_video_url])
    meta = ydl.extract_info(
        lyrics_video_url, download=False)
    
    shutil.move(os.path.join(download_dir, f"{meta['id']}.mp4"), os.path.join(download_dir, "lyrics_video.mp4"))
    
    for i, url in enumerate(stage_video_urls):
        ydl.download([url])
        meta = ydl.extract_info(
            url, download=False)
        
        shutil.move(os.path.join(download_dir, f"{meta['id']}.mp4"), os.path.join(download_dir, f"stage_video{i}.mp4"))

[youtube] mIXR-7u2Kas: Downloading webpage
[youtube] mIXR-7u2Kas: Downloading video info webpage
[download] /home/sean/Documents/experiments/stage-mix-generator/temp/mIXR-7u2Kas.mp4 has already been downloaded
[download] 100% of 8.88MiB
[youtube] mIXR-7u2Kas: Downloading webpage
[youtube] mIXR-7u2Kas: Downloading video info webpage
[youtube] kda_53A0ZpI: Downloading webpage
[youtube] kda_53A0ZpI: Downloading video info webpage
[download] /home/sean/Documents/experiments/stage-mix-generator/temp/kda_53A0ZpI.mp4 has already been downloaded
[download] 100% of 25.29MiB
[youtube] kda_53A0ZpI: Downloading webpage
[youtube] kda_53A0ZpI: Downloading video info webpage
[youtube] zzq8uI1BnJk: Downloading webpage
[youtube] zzq8uI1BnJk: Downloading video info webpage
[download] /home/sean/Documents/experiments/stage-mix-generator/temp/zzq8uI1BnJk.mp4 has already been downloaded
[download] 100% of 21.99MiB
[youtube] zzq8uI1BnJk: Downloading webpage
[youtube] zzq8uI1BnJk: Downloading video info webp

In [3]:
lyrics_video_path = os.path.join(download_dir, 'lyrics_video.mp4')
stage_video_paths = [ os.path.join(download_dir, f'stage_video{i}.mp4') for i in range(len(stage_video_urls)) ]

# find timestamp of start of song in stage videos


# get subclips of the song in the stage videos

In [ ]:
# every 5 seconds during the song, splice clips from different performances